In [ ]:
from deltalake.writer import write_deltalake
from deltalake import DeltaTable
import duckdb
import pandas
import os
from dotenv import load_dotenv

In [2]:
con = duckdb.connect()

In [ ]:
# Carrega as variáveis do arquivo .env
load_dotenv()

# Carrega as variáveis do arquivo .env
load_dotenv()

# Acessa as credenciais
azure_storage_account_name = os.getenv('AZURE_STORAGE_ACCOUNT_NAME')
azure_storage_access_key = os.getenv('AZURE_STORAGE_ACCESS_KEY')
azure_storage_client_id = os.getenv('AZURE_STORAGE_CLIENT_ID')
azure_storage_client_secret = os.getenv('AZURE_STORAGE_CLIENT_SECRET')
azure_storage_tenant_id = os.getenv('AZURE_STORAGE_TENANT_ID')

In [ ]:
storage_options = {
    'AZURE_STORAGE_ACCOUNT_NAME': azure_storage_account_name,
    'AZURE_STORAGE_ACCESS_KEY': azure_storage_access_key,
    'AZURE_STORAGE_CLIENT_ID': azure_storage_client_id,
    'AZURE_STORAGE_CLIENT_SECRET': azure_storage_client_secret, 
    'AZURE_STORAGE_TENANT_ID': azure_storage_tenant_id
}


### Carga Completa arquivos

### Funções

In [4]:
def ler_delta_bronze(table_name):
    uri= f'az://bronze/vendas/{table_name}'
    
    dt = DeltaTable(uri, storage_options=storage_options)
    return dt.to_pandas()

In [5]:
def ler_delta_silver(table_name):
    uri= f'az://silver/vendas/{table_name}'
    
    dt = DeltaTable(uri, storage_options=storage_options)
    return dt.to_pandas()

In [6]:
def escreve_delta_silver(df,table_name, modoEscrita):
    uri= f'az://silver/vendas/{table_name}'
    write_deltalake( 
        uri,
        df,
        mode=modoEscrita,
        storage_options=storage_options
    )

### Lendo da Bronze

In [7]:
brands = ler_delta_bronze('brands')
categories = ler_delta_bronze('categories')
customers = ler_delta_bronze('customers')
order_items = ler_delta_bronze('order_items')
orders = ler_delta_bronze('orders')
products = ler_delta_bronze('products')
staffs = ler_delta_bronze('staffs')
stocks = ler_delta_bronze('stocks')
stores = ler_delta_bronze('stores')

### Carregando dados para Silver

#### Order Sales

In [12]:
dtl_orders_sales = ler_delta_silver('order_sales')

In [15]:
# TODO fazer inserção tabela inteira quando não tiver dtl_orders_sales
order_sales = con.sql("""
    WITH order_sales_bronze AS (
    SELECT
        P.product_id,
        P.product_name,
        B.brand_name,
        CT.category_name,
        C.customer_id,
        C.first_name || C.last_name AS customer_name,
        S.staff_id,
        S.first_name || S.last_name AS staff_name,
        ST.store_id,
        ST.store_name,
        OI.order_id,
        OI.item_id,
        O.order_date,
        OI.quantity,
        OI.list_price,
        OI.discount
    FROM order_items OI
    LEFT JOIN orders O ON OI.order_id = O.order_id
    LEFT JOIN products P ON P.product_id = OI.product_id
    LEFT JOIN brands B ON P.brand_id = B.brand_id
    LEFT JOIN categories CT ON P.category_id = CT.category_id
    LEFT JOIN customers C ON O.customer_id = C.customer_id
    LEFT JOIN staffs S ON O.staff_id = S.staff_id
    LEFT JOIN stores ST ON ST.store_id = O.store_id
    ),
    dt_order_sales AS (
    SELECT
        MAX(order_date) AS order_date
    FROM dtl_orders_sales     
    )
    SELECT
        *
    FROM order_sales_bronze
    WHERE order_date > (
        SELECT 
            order_date
        FROM dt_order_sales
                      )
    """).to_df()
if len(order_sales) > 0:
    escreve_delta_silver(order_sales,'order_sales','append')

,product_id,product_name,brand_name,category_name,customer_id,customer_name,staff_id,staff_name,store_id,store_name,order_id,item_id,order_date,quantity,list_price,discount


#### Stocks

In [9]:
stocks_snapshot  = con.sql("""
    SELECT
        *,
        current_date as dt_stock
    FROM stocks
    """).to_df()
escreve_delta_silver(stocks_snapshot,'stocks_snapshot','append')

In [16]:
stocks_snapcho

,store_id,product_id,quantity,dt_stocks
0,1,1,27,2024-09-09
1,1,2,5,2024-09-09
2,1,3,6,2024-09-09
3,1,4,23,2024-09-09
4,1,5,22,2024-09-09
...,...,...,...,...
934,3,309,30,2024-09-09
935,3,310,8,2024-09-09
936,3,311,23,2024-09-09
937,3,312,18,2024-09-09


## Fecha conexões

In [55]:
con.close()